In [1]:
import requests
import yaml
import time
from tqdm import tqdm
from datetime import datetime
import steamreviews
from time import sleep
import os
import json
os.chdir("C:\\Users\weejw\Documents\steamRecommendation")
from db_config.db_funtion import db_execute


In [2]:
def review_insert(reviews, appid, review_type):
    need_data = ['votes_up', 'votes_funny', 'steam_purchase', 'received_for_free', 'written_during_early_access', 'review']
    need_data_dic ={'steam_appid': appid}
        
    for review in reviews['reviews']:
        author = review['author']
        steamid = author['steamid']
        sql = "SELECT id FROM review WHERE steam_appid=%s AND steamid='%s'" % (appid, steamid)
        if db_function.execute_select(sql):
            return
        for k, v in author.items():
            if k == 'last_played':
                v = datetime.fromtimestamp(v).strftime("%Y-%m-%d %H:%M:%S")
            need_data_dic[k] = v
        
        for col in need_data:
            need_data_dic[col] = review[col]
            
        
        need_data_dic['review_type'] = review_type
        
        placeholders = ', '.join(['%s'] * len(need_data_dic))
        columns = ', '.join(need_data_dic.keys())
        
        sql = "INSERT IGNORE INTO %s ( %s ) VALUES ( %s )" % ('review', columns, placeholders)
        db_function.execute_insert(sql, list(need_data_dic.values()))


In [ ]:
start_time = time.time()
try_cnt = 0
db_function = None

start_time = time.time()
try_cnt = 0

with open('C:\\Users\weejw\Documents\steamRecommendation\db_config\db_auth.yaml', 'r') as file:
    db_auth = yaml.safe_load(file)

try:
    db_function = db_execute(db_auth['local_db']["host"], 
                             db_auth['local_db']["port"],
                             db_auth['local_db']["user"],
                             db_auth['local_db']["db"], 
                             db_auth['local_db']["passwd"])

    sql = "SELECT steam_appid FROM game_info"
    app_ids = db_function.execute_select(sql)
    for appid in tqdm(app_ids):
        try:
            try_cnt +=1
            current_time = time.time()
            elapsed_time = current_time - start_time
            if try_cnt > 200 and elapsed_time < 300:
                sleep(60)
                try_cnt = 0
                start_time = time.time()

            appid = appid[0]
            
            review_type='negative'
            num_per_page=100
            day_range=100
            language='korean'
            
            
            url = f"https://store.steampowered.com/appreviews/{appid}?json=1&review_type={review_type}&day_range={day_range}&num_per_page={num_per_page}&language={language}"
            resp = requests.get(url)
            
            if not resp.ok:
                print(url, resp)
            reviews = resp.json()
            
            if reviews['reviews']:
                review_insert(reviews, appid, review_type)

            sleep(5)

            
        except KeyboardInterrupt as e:
            break
        
        except json.decoder.JSONDecodeError:
                continue
        
except:
    raise
    
finally:
    db_function.curs.close()
    db_function.conn.close()
    

  0%|▎                                                                                                                                                                                                                                           | 4/3518 [00:21<5:12:17,  5.33s/it]